In [37]:
import os
import numpy as np
import random
from environment import Environment
import copy
import configs
import pickle
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
import ollama
from collections import defaultdict
from collections import Counter

In [38]:
# negotiator를 구성하기 위한 union find 알고리즘
def find(parent, i):
    if parent[i] == i:
        return i
    else:
        return find(parent, parent[i])

def union(parent, rank, x, y):
    xroot = find(parent, x)
    yroot = find(parent, y)

    if xroot != yroot:
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

def merge_sets(lists):
    element_to_index = {}
    for i, subset in enumerate(lists):
        for element in subset:
            element_to_index[element] = i

    parent = [i for i in range(len(lists))]
    rank = [0] * len(lists)

    for subset in lists:
        first_element = subset[0]
        for element in subset[1:]:
            union(parent, rank, find(parent, element_to_index[first_element]), find(parent, element_to_index[element]))

    new_sets = {}
    for element in element_to_index:
        root = find(parent, element_to_index[element])
        if root not in new_sets:
            new_sets[root] = set()
        new_sets[root].add(element)

    return [list(s) for s in new_sets.values()]


def determine_direction(x, y, x_finish, y_finish):

    dx = int(x_finish) - x
    dy = int(y_finish) - y
    
    if dx > 0 and dy > 0:
        return "northeast"
    elif dx > 0 and dy < 0:
        return "southeast"
    elif dx < 0 and dy > 0:
        return "northwest"
    elif dx < 0 and dy < 0:
        return "southwest"
    elif dx > 0 and dy == 0:
        return "east"
    elif dx < 0 and dy == 0:
        return "west"
    elif dx == 0 and dy > 0:
        return "north"
    elif dx == 0 and dy < 0:
        return "south"
    else:
        return "same location"
    
def get_direction(center, max_position):
    
    row_diff = center[0] - max_position[0]
    col_diff = center[1] - max_position[1]

    if row_diff > 0 and col_diff > 0:
        return "northwest"
    elif row_diff > 0 and col_diff == 0:
        return "north"
    elif row_diff > 0 and col_diff < 0:
        return "northeast"
    elif row_diff == 0 and col_diff > 0:
        return "west"
    elif row_diff == 0 and col_diff < 0:
        return "east"
    elif row_diff < 0 and col_diff > 0:
        return "southwest"
    elif row_diff < 0 and col_diff == 0:
        return "south"
    elif row_diff < 0 and col_diff < 0:
        return "southeast"


# 방향 정의
directiondict = {
    'stay': 0, 'north': 1, 'south': 2, 'west': 3, 'east': 4
}

In [39]:
# 프롬프트
class gpt4pathfinding:
    def heuristic_navigate(self, agent, heuristic_direction, east, west, north, south, high_priority_agent):
        response = ollama.generate(
        model="llama3",
        prompt=f"""
                <s>[INST] <<SYS>>
                You're a directional path finder - you know which path to take to reach your goal.
                <</SYS>>
                You can only make one move at a time.
                Your actions have 4 options: east, west, south, north.
                
                {east}
                {west}
                {south}
                {north}

                The locations of other potentially conflicting agents are as follows.
                {high_priority_agent}

                Your map says going {heuristic_direction} will shorten the distance to the destination.
                Of the directions presented, choose the one that minimizes conflict with other potentially conflicting agents.
                
                You decide which direction to move next.

                Below are examples of answers.
                Answer with only one word.

                EXAMPLE 1

                south

                EXAMPLE 2

                north
                [/INST]
                """
        )
        return agent, response['response'].lower()
    
    def give_way(self, east, west, north, south, prioragentdirection, goaldirection):
        response = ollama.generate(
        model="llama3",
        prompt=f"""
                <s>[INST] <<SYS>>
                You're a directional path finder - you know which path to take to reach your goal.
                <</SYS>>

                The position of the prior path finder is {prioragentdirection} relative to you.

                You MUST avoid collisions with prior path finder.

                Your actions have five options: east, west, south, north, and stay.

                {east}
                {west}
                {south}
                {north}

                It is recommended to move away from the prior agent.
                It is recommended to going to the 90-degree turn rather than the other side of the prior pathfinder, but you can ignore this instruction depending on your situation.

                You decide which direction to move next.

                Below are examples of answers.
                Answer with only one word.

                EXAMPLE 1

                south

                EXAMPLE 2

                north
                """
        )
        return response['response'].lower()
    
pathfinder = gpt4pathfinding()

In [40]:
def run_one_example(example, max_steps):

  num_agents = len(example[1])
  map_width = len(example[0]) - 1

  env = Environment()
  env.load(np.array(example[0]), np.array(example[1]), np.array(example[2]))

  env.initialize_perceived_maps()
  env.get_perceived_heuri_map()

  def parameter_navigate(i):

    observe = env.observe()

    a_obs = observe[0][i][1]

    if a_obs[4][3] == 0:
      west = """To your west lies a path."""
    else:
      west = """Your west is blocked by a wall."""

    if a_obs[4][5] == 0:
      east = """To your east lies a path."""
    else:
      east = """Your east is blocked by a wall."""

    if a_obs[5][4] == 0:
      south = """To your south lies a path."""
    else:
      south = """Your south is blocked by a wall."""

    if a_obs[3][4] == 0:
      north = """To your north lies a path."""
    else:
      north = """Your north is blocked by a wall."""

    directions = []

    if env.observe()[0][i][6][4, 4] == 1:
      directions.append('north')
    if env.observe()[0][i][7][4, 4] == 1:
      directions.append('south')
    if env.observe()[0][i][8][4, 4] == 1:
      directions.append('west')
    if env.observe()[0][i][9][4, 4] == 1:
      directions.append('east')
    
    heuristic_direction = directions

    agent_map = env.observe()[0][i][0][2:7, 2:7]
    positions = np.argwhere(agent_map > i+1)
    center = (2, 2)
    directions = [get_direction(center, pos) for pos in positions]
    direction_counts = Counter(directions)
    direction_messages = [f"There are {count} obstacles to the {direction}." for direction, count in direction_counts.items() if count > 0]
    high_priority_agent = '\n'.join(direction_messages)

    return heuristic_direction, east, west, north, south, high_priority_agent
  
  def parameter_give_way(i):

    observe = env.observe()

    a_obs = observe[0][i][1]
    x = observe[1][i][1]
    y = map_width-observe[1][i][0]
    x_finish = example[2][i][1]
    y_finish = map_width-example[2][i][0]
    goaldirection = determine_direction(x, y, x_finish, y_finish)

    if a_obs[4][3] == 0:
        west = """To your west lies a path."""
    else:
        west = """Your west is blocked by a wall."""

    if a_obs[4][5] == 0:
        east = """To your east lies a path."""
    else:
        east = """Your east is blocked by a wall."""

    if a_obs[5][4] == 0:
        south = """To your south lies a path."""
    else:
        south = """Your south is blocked by a wall."""

    if a_obs[3][4] == 0:
        north = """To your north lies a path."""
    else:
        north = """Your north is blocked by a wall."""


    # prior agent의 direction 구하기
    agent_map = env.observe()[0][i][0][2:7, 2:7]
    center = (2, 2)
    max_position = np.unravel_index(np.argmax(agent_map), agent_map.shape)

    row_diff = center[0] - max_position[0]
    col_diff = center[1] - max_position[1]

    if row_diff > 0 and col_diff > 0:
        prioragentdirection = "northwest"
    elif row_diff > 0 and col_diff == 0:
        prioragentdirection = "north"
    elif row_diff > 0 and col_diff < 0:
        prioragentdirection = "northeast"
    elif row_diff == 0 and col_diff > 0:
        prioragentdirection = "west"
    elif row_diff == 0 and col_diff < 0:
        prioragentdirection = "east"
    elif row_diff < 0 and col_diff > 0:
        prioragentdirection = "southwest"
    elif row_diff < 0 and col_diff == 0:
        prioragentdirection = "south"
    elif row_diff < 0 and col_diff < 0:
        prioragentdirection = "southeast"

    return east, west, north, south, prioragentdirection, goaldirection

  
  #에이전트들의 지난번 액션들과 좌표
  action_list = [[] for _ in range(num_agents)]
  coordinate_list = [[] for _ in range(num_agents)]
  step_list = [0 for _ in range(num_agents)]
  dead_end_state = [0 for _ in range(num_agents)]
  steps = 0

  # turn
  while steps < max_steps:

    for i in range(num_agents):
      if not np.array_equal(env.agents_pos[i], env.goals_pos[i]):
        step_list[i] += 1

    for i in range(num_agents):
      x_coordinate = env.observe()[1][i][1]
      y_coordinate = map_width-env.observe()[1][i][0]

      coordinate_list[i].append([x_coordinate, y_coordinate])

    step = [0 for i in range(num_agents)]

    #각 에이전트들의 시야에 있는 자신과 다른 에이전트들
    FOV_agents = []
    for i in range(num_agents):
      if np.any(env.observe()[0][i][0][2:7, 2:7]):
        non_zero_elements = env.observe()[0][i][0][2:7, 2:7][env.observe()[0][i][0][2:7, 2:7] != 0].tolist()
        non_zero_elements = [element - 1 for element in non_zero_elements]
        non_zero_elements.append(i)
        FOV_agents.append(non_zero_elements)

    #알고리즘을 이용해 연결된 집합 찾기
    connected_sets = merge_sets(FOV_agents)

    #연결이 있는 모든 에이전트들
    deadlocked_agents = [item for sublist in connected_sets for item in sublist]

    for i in range(num_agents):

      #교착상태에 빠지지 않은 에이전트는 독립적으로 navigate
      if i not in deadlocked_agents:

        if np.array_equal(env.agents_pos[i], env.goals_pos[i]):
          step[i] = directiondict['stay']
          continue

        heuristic_direction, east, west, north, south, high_priority_agent = parameter_navigate(i)
        try:
          direction = random.choice(heuristic_direction)
        except:
          direction = 'stay'

        step[i] = directiondict[direction]

    
    # 교착상태에 빠진 에이전트에 대해
    for connected_set in connected_sets:
      
      results = []

      for i in connected_set: # 여기서 i는 agent의 번호들

        if np.array_equal(env.agents_pos[i], env.goals_pos[i]):
          direction = 'stay'
          results.append(direction)
          continue
      
      #각각의 에이전트에 대해 원래 계획된 경로를 계산

        heuristic_direction, east, west, north, south, high_priority_agent = parameter_navigate(i)

        _, result = pathfinder.heuristic_navigate(i, heuristic_direction, east, west, north, south, high_priority_agent)

        direction = result

        if direction not in directiondict:
          direction = 'stay'

        results.append(direction)

      env_copy = copy.deepcopy(env)
      planned_step = [0 for _ in range(num_agents)]
      for idx, agent_idx in enumerate(connected_set):
        planned_step[agent_idx] = directiondict[results[idx]]
      if -0.5 in env_copy.step(planned_step)[1]:
        collision = True
      else:
        collision = False

      if not collision:
        for idx, agent_idx in enumerate(connected_set):
          action_list[agent_idx].append(results[idx])
          step[agent_idx] = directiondict[results[idx]]
      else:
      # 먼저 prior agent는 원래대로 이동하게 함
        prior_agent_number = max(connected_set)
        prior_agent_idx = connected_set.index(prior_agent_number)
        action_list[prior_agent_number].append(results[prior_agent_idx])
        step[prior_agent_number] = directiondict[results[prior_agent_idx]]
        for i in connected_set:
          if i != prior_agent_number:
              
              east, west, north, south, prioragentdirection, goaldirection = parameter_give_way(i)

              result = pathfinder.give_way(east, west, north, south, prioragentdirection, goaldirection)

              direction = result

              if direction not in directiondict:
                direction = 'stay'

              action_list[i] = []
      
              step[i] = directiondict[direction]


    observation, rewards, done, info = env.step(step)

    env.update_perceived_maps()
    env.get_perceived_heuri_map()

    if done:
      break
    steps += 1

  return step_list

In [41]:
def simulate_all_tests(tests, max_steps):
    all_step_lists = []
    for example in tests:
        step_list = run_one_example(example, max_steps)
        all_step_lists.append(step_list)
        print(step_list)
    return all_step_lists

In [42]:
with open('./test_set/{}_{}agents.pth'.format('warehouse', 4), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 512)

[65, 73, 101, 72]
[64, 142, 142, 87]
[54, 21, 99, 39]
[38, 34, 152, 132]
[147, 141, 165, 125]
[17, 3, 48, 152]
[100, 87, 46, 14]
[87, 80, 158, 162]
[33, 7, 62, 45]
[105, 156, 53, 102]
[150, 72, 85, 67]
[104, 53, 107, 28]
[54, 128, 151, 159]
[70, 73, 59, 19]
[159, 39, 3, 38]
[149, 38, 50, 75]
[176, 174, 76, 65]
[52, 70, 160, 25]
[49, 43, 162, 50]
[40, 34, 166, 77]


In [43]:
with open('./test_set/{}_{}agents.pth'.format('warehouse', 8), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 512)

[60, 88, 9, 104, 144, 42, 25, 142]
[87, 74, 69, 84, 137, 64, 57, 10]
[53, 111, 59, 65, 30, 42, 49, 153]
[91, 30, 41, 148, 65, 32, 128, 60]
[17, 132, 129, 95, 19, 16, 109, 96]
[124, 39, 103, 45, 38, 13, 79, 68]
[6, 20, 67, 128, 146, 59, 102, 61]
[94, 33, 69, 22, 72, 22, 9, 69]
[512, 512, 43, 192, 64, 53, 93, 27]
[151, 51, 79, 10, 56, 85, 21, 123]
[82, 106, 10, 127, 131, 30, 155, 19]
[83, 86, 46, 45, 130, 62, 27, 6]
[157, 37, 193, 82, 67, 92, 100, 30]
[41, 23, 140, 134, 82, 37, 121, 102]
[113, 113, 43, 42, 87, 142, 39, 40]
[30, 50, 128, 151, 154, 117, 132, 62]
[34, 20, 98, 180, 18, 127, 44, 53]
[64, 40, 63, 156, 34, 100, 69, 22]
[41, 148, 64, 97, 152, 109, 164, 98]
[145, 195, 156, 74, 64, 131, 61, 156]


In [44]:
with open('./test_set/{}_{}agents.pth'.format('warehouse', 16), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 512)

[121, 77, 143, 143, 81, 10, 144, 66, 63, 45, 70, 77, 63, 63, 98, 116]
[162, 31, 126, 175, 98, 43, 7, 157, 120, 13, 154, 37, 40, 16, 77, 13]
[148, 123, 60, 12, 22, 40, 114, 44, 171, 112, 41, 134, 49, 3, 106, 97]
[112, 47, 122, 77, 51, 31, 100, 113, 110, 43, 87, 147, 40, 158, 73, 168]
[156, 101, 103, 110, 203, 115, 34, 115, 83, 63, 162, 129, 49, 79, 93, 62]
[21, 109, 153, 37, 117, 34, 158, 147, 69, 32, 43, 70, 93, 90, 64, 118]
[49, 158, 90, 172, 72, 125, 70, 138, 60, 122, 72, 158, 30, 112, 36, 70]
[153, 139, 148, 134, 120, 79, 61, 72, 145, 70, 17, 48, 45, 157, 124, 60]
[55, 38, 14, 131, 78, 98, 68, 156, 139, 47, 33, 157, 130, 81, 70, 89]
[203, 22, 9, 146, 20, 55, 170, 38, 133, 57, 94, 93, 138, 48, 85, 92]
[164, 146, 80, 177, 150, 119, 128, 229, 158, 174, 97, 185, 32, 88, 96, 141]
[137, 111, 25, 144, 137, 68, 140, 150, 67, 102, 193, 20, 93, 39, 82, 58]
[108, 57, 26, 54, 126, 119, 147, 126, 171, 33, 153, 17, 145, 95, 104, 40]
[13, 74, 142, 36, 36, 44, 29, 74, 148, 80, 42, 182, 149, 122, 13

In [45]:
with open('./test_set/{}_{}agents.pth'.format('warehouse', 32), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 512)

[151, 56, 42, 36, 39, 68, 134, 104, 102, 45, 124, 128, 161, 68, 133, 62, 6, 197, 124, 105, 101, 122, 114, 29, 25, 47, 114, 102, 34, 119, 135, 104]
[130, 72, 94, 183, 15, 126, 154, 46, 214, 38, 163, 155, 21, 148, 188, 92, 125, 35, 62, 138, 166, 128, 142, 114, 79, 164, 79, 17, 34, 84, 36, 8]
[94, 151, 62, 14, 226, 95, 73, 119, 28, 51, 101, 37, 114, 78, 151, 65, 168, 110, 129, 152, 91, 130, 103, 67, 191, 65, 104, 64, 31, 37, 62, 53]
[166, 107, 54, 136, 172, 95, 110, 102, 137, 96, 170, 106, 45, 161, 164, 68, 40, 33, 176, 88, 122, 37, 191, 75, 169, 89, 162, 149, 38, 20, 123, 50]
[60, 27, 136, 15, 91, 222, 152, 165, 62, 151, 50, 181, 52, 71, 140, 75, 75, 104, 83, 83, 54, 166, 130, 142, 86, 154, 130, 97, 129, 23, 122, 99]
[307, 41, 138, 39, 159, 162, 51, 512, 11, 103, 80, 165, 64, 178, 162, 509, 48, 167, 103, 142, 181, 36, 92, 12, 54, 107, 77, 67, 19, 102, 108, 152]
[149, 180, 133, 9, 79, 122, 74, 7, 75, 120, 61, 48, 40, 28, 151, 49, 40, 11, 102, 66, 28, 14, 51, 15, 26, 121, 60, 170, 90, 93, 

In [46]:
with open('./test_set/{}_{}agents.pth'.format('warehouse', 64), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 512)

[83, 171, 264, 182, 78, 124, 512, 105, 167, 512, 39, 147, 95, 81, 57, 32, 112, 76, 117, 132, 59, 72, 54, 142, 165, 160, 105, 165, 80, 28, 166, 231, 130, 122, 87, 149, 178, 48, 91, 112, 45, 132, 49, 90, 124, 95, 87, 150, 15, 99, 45, 137, 65, 137, 68, 163, 58, 122, 63, 49, 41, 96, 151, 27]
[109, 15, 46, 102, 95, 37, 436, 1, 156, 23, 91, 72, 167, 134, 191, 55, 59, 104, 101, 147, 62, 21, 114, 62, 51, 125, 97, 82, 83, 71, 115, 45, 8, 171, 152, 108, 70, 113, 52, 143, 160, 94, 111, 84, 37, 94, 129, 8, 103, 40, 173, 116, 170, 83, 175, 173, 85, 64, 161, 154, 36, 31, 126, 107]
[45, 104, 172, 123, 235, 78, 94, 134, 123, 144, 162, 30, 167, 147, 110, 85, 164, 64, 22, 20, 182, 48, 246, 117, 163, 104, 40, 37, 128, 190, 50, 80, 85, 43, 98, 8, 79, 51, 151, 50, 24, 61, 175, 87, 150, 160, 31, 70, 88, 151, 45, 170, 94, 35, 150, 21, 23, 90, 64, 37, 80, 96, 65, 120]
[141, 178, 37, 226, 200, 50, 22, 94, 124, 93, 46, 89, 192, 92, 75, 28, 91, 55, 24, 95, 151, 137, 99, 177, 128, 17, 59, 7, 32, 45, 112, 200, 512

In [47]:
with open('./test_set/{}_{}agents.pth'.format('den312d', 4), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[132, 48, 67, 9]
[78, 81, 31, 76]
[41, 47, 94, 57]
[75, 47, 51, 7]
[34, 69, 46, 37]
[30, 27, 90, 56]
[32, 49, 37, 57]
[35, 32, 10, 77]
[51, 13, 88, 42]
[97, 60, 74, 42]
[59, 56, 107, 58]
[43, 41, 54, 35]
[58, 72, 28, 16]
[50, 22, 64, 41]
[78, 16, 84, 36]
[56, 30, 13, 47]
[80, 64, 31, 20]
[38, 78, 34, 29]
[57, 45, 89, 28]
[49, 79, 31, 44]


In [48]:
with open('./test_set/{}_{}agents.pth'.format('den312d', 8), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[13, 29, 21, 83, 67, 62, 84, 69]
[68, 71, 37, 16, 88, 72, 30, 31]
[51, 92, 28, 37, 62, 61, 114, 67]
[33, 24, 4, 45, 10, 77, 66, 53]
[65, 96, 90, 12, 43, 61, 81, 61]
[92, 66, 44, 36, 37, 84, 85, 52]
[27, 34, 88, 36, 28, 51, 19, 85]
[78, 46, 9, 90, 39, 71, 111, 65]
[31, 64, 10, 56, 106, 71, 61, 18]
[49, 78, 101, 70, 81, 82, 14, 76]
[53, 34, 69, 38, 82, 2, 48, 51]
[118, 84, 42, 68, 90, 19, 41, 64]
[81, 36, 39, 64, 17, 16, 15, 76]
[11, 40, 71, 4, 82, 52, 68, 47]
[78, 98, 64, 24, 28, 79, 70, 18]
[59, 34, 29, 5, 37, 72, 87, 46]
[34, 88, 19, 76, 101, 23, 81, 46]
[80, 72, 59, 74, 79, 20, 19, 84]
[45, 33, 99, 71, 70, 85, 8, 71]
[58, 68, 98, 18, 68, 28, 37, 102]


In [49]:
with open('./test_set/{}_{}agents.pth'.format('den312d', 16), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[43, 52, 44, 71, 49, 36, 111, 133, 29, 13, 77, 76, 7, 76, 52, 79]
[16, 96, 92, 85, 8, 130, 130, 144, 104, 80, 59, 103, 65, 92, 88, 80]
[46, 133, 26, 56, 3, 87, 20, 88, 85, 66, 70, 37, 73, 55, 25, 32]
[45, 85, 85, 40, 57, 78, 65, 64, 53, 17, 88, 43, 29, 101, 45, 27]
[71, 117, 76, 54, 75, 25, 42, 41, 57, 59, 59, 71, 39, 41, 86, 51]
[18, 96, 66, 39, 80, 38, 96, 9, 92, 48, 53, 64, 39, 84, 41, 15]
[53, 72, 5, 50, 29, 67, 63, 45, 24, 23, 74, 36, 68, 98, 54, 41]
[77, 63, 55, 84, 73, 64, 12, 55, 37, 16, 25, 51, 36, 15, 5, 120]
[62, 21, 80, 63, 69, 79, 38, 74, 86, 60, 27, 78, 75, 28, 79, 28]
[23, 56, 33, 52, 116, 28, 74, 95, 37, 31, 37, 124, 43, 103, 95, 4]
[82, 3, 17, 69, 65, 62, 83, 52, 30, 40, 41, 43, 67, 34, 101, 108]
[51, 34, 43, 29, 35, 142, 57, 4, 69, 105, 42, 30, 34, 66, 43, 20]
[30, 42, 49, 34, 102, 7, 55, 99, 44, 67, 14, 19, 49, 44, 34, 90]
[81, 5, 19, 55, 7, 41, 61, 43, 38, 43, 123, 88, 53, 51, 46, 66]
[48, 111, 60, 88, 61, 54, 71, 67, 100, 36, 95, 87, 14, 59, 102, 77]
[56, 114, 16, 

In [50]:
with open('./test_set/{}_{}agents.pth'.format('den312d', 32), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[117, 89, 36, 61, 19, 16, 69, 79, 108, 90, 66, 13, 97, 36, 89, 59, 10, 76, 37, 45, 122, 95, 36, 14, 59, 78, 45, 24, 163, 105, 41, 21]
[67, 56, 103, 55, 18, 120, 68, 90, 78, 42, 26, 61, 86, 72, 75, 65, 104, 101, 59, 53, 28, 30, 37, 112, 53, 6, 91, 32, 4, 42, 20, 89]
[78, 27, 17, 14, 43, 89, 136, 120, 92, 92, 138, 40, 96, 6, 23, 170, 30, 113, 25, 40, 62, 136, 20, 79, 43, 11, 99, 87, 85, 96, 57, 87]
[7, 97, 36, 107, 75, 51, 51, 20, 53, 101, 115, 103, 33, 109, 70, 96, 44, 98, 75, 51, 49, 13, 54, 41, 102, 39, 32, 69, 100, 14, 79, 49]
[14, 10, 39, 95, 74, 59, 254, 124, 84, 173, 35, 11, 101, 47, 85, 114, 256, 36, 57, 101, 80, 87, 74, 58, 25, 20, 30, 30, 67, 16, 83, 58]
[91, 20, 158, 158, 42, 60, 182, 126, 92, 15, 97, 63, 97, 130, 142, 51, 87, 22, 43, 124, 55, 32, 81, 35, 25, 87, 72, 26, 65, 101, 66, 40]
[89, 42, 96, 7, 79, 129, 54, 50, 32, 97, 58, 81, 130, 75, 56, 52, 29, 92, 32, 115, 7, 47, 12, 61, 54, 105, 4, 5, 40, 54, 45, 68]
[62, 78, 121, 138, 67, 92, 117, 41, 3, 41, 90, 77, 108, 75, 79,

In [51]:
with open('./test_set/{}_{}agents.pth'.format('den312d', 64), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[195, 85, 129, 135, 128, 100, 137, 102, 124, 9, 17, 94, 43, 125, 91, 60, 115, 161, 56, 171, 139, 116, 176, 73, 148, 17, 80, 53, 156, 38, 126, 150, 71, 130, 108, 67, 54, 97, 12, 95, 48, 76, 38, 157, 98, 62, 8, 160, 124, 65, 28, 50, 122, 103, 17, 97, 25, 29, 83, 47, 8, 104, 57, 79]
[8, 50, 15, 7, 204, 91, 35, 87, 198, 41, 89, 41, 168, 85, 19, 33, 145, 11, 66, 120, 60, 38, 123, 84, 33, 123, 104, 83, 104, 47, 104, 33, 7, 160, 69, 139, 73, 50, 61, 122, 121, 113, 152, 32, 115, 58, 125, 146, 43, 108, 74, 52, 60, 56, 22, 26, 80, 33, 72, 22, 32, 35, 87, 35]
[32, 33, 175, 67, 53, 147, 72, 174, 52, 61, 87, 28, 44, 32, 46, 111, 69, 108, 33, 170, 39, 73, 166, 89, 120, 18, 53, 161, 71, 117, 81, 178, 147, 144, 99, 40, 82, 71, 38, 70, 11, 83, 147, 14, 75, 4, 43, 45, 51, 33, 25, 43, 17, 33, 96, 51, 24, 60, 40, 24, 44, 14, 41, 84]
[75, 47, 69, 224, 65, 70, 110, 193, 159, 62, 134, 71, 138, 172, 118, 203, 129, 99, 178, 94, 4, 134, 146, 70, 83, 50, 90, 193, 147, 45, 110, 195, 126, 51, 86, 136, 53, 88, 116,

In [16]:
def add_walls(array):
    rows, cols = array.shape
    new_array = np.ones((rows + 2, cols + 2), dtype=array.dtype)
    new_array[1:-1, 1:-1] = array
    return new_array

def simulate_all_tests(tests, max_steps):
    all_step_lists = []
    for example in tests:
        example = (add_walls(np.array(example[0])), np.array([(x + 1, y + 1) for (x, y) in example[1]]), np.array([(x + 1, y + 1) for (x, y) in example[2]]))
        step_list = run_one_example(example, max_steps)
        all_step_lists.append(step_list)
        print(step_list)
    return all_step_lists

In [24]:
with open('./test_set/{}_{}agents.pth'.format('random64', 4), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[26, 53, 10, 57]
[38, 65, 55, 52]
[53, 41, 41, 38]
[93, 26, 31, 41]
[45, 29, 85, 22]
[49, 55, 67, 22]
[34, 34, 49, 86]
[28, 35, 46, 20]
[33, 34, 42, 32]
[33, 50, 19, 13]
[64, 40, 22, 30]
[50, 57, 67, 80]
[42, 73, 3, 51]
[65, 44, 55, 6]
[33, 51, 28, 35]
[26, 29, 35, 41]
[48, 39, 87, 45]
[17, 57, 38, 46]
[61, 55, 53, 71]
[47, 32, 42, 5]


In [25]:
with open('./test_set/{}_{}agents.pth'.format('random64', 8), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[44, 55, 78, 56, 33, 56, 34, 38]
[79, 42, 62, 51, 35, 36, 105, 26]
[15, 73, 42, 50, 25, 13, 40, 57]
[47, 16, 65, 31, 33, 47, 31, 45]
[21, 8, 69, 51, 57, 32, 93, 54]
[63, 21, 68, 48, 36, 34, 9, 44]
[76, 52, 5, 38, 28, 57, 40, 77]
[76, 59, 77, 59, 21, 93, 72, 50]
[74, 7, 28, 39, 24, 40, 50, 36]
[35, 21, 27, 47, 75, 31, 29, 27]
[68, 75, 47, 88, 20, 73, 33, 31]
[44, 29, 72, 17, 52, 56, 35, 83]
[14, 28, 94, 34, 18, 14, 43, 36]
[31, 42, 39, 20, 38, 67, 10, 81]
[43, 54, 32, 80, 101, 43, 50, 13]
[67, 37, 69, 12, 45, 19, 54, 21]
[38, 71, 60, 51, 50, 30, 46, 26]
[2, 39, 64, 11, 12, 56, 64, 29]
[35, 71, 59, 61, 54, 20, 48, 13]
[24, 34, 71, 45, 21, 54, 70, 37]


In [26]:
with open('./test_set/{}_{}agents.pth'.format('random64', 16), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[46, 27, 40, 36, 38, 38, 41, 57, 11, 28, 56, 29, 33, 24, 38, 33]
[32, 23, 74, 49, 45, 87, 59, 43, 38, 49, 101, 16, 41, 41, 18, 64]
[50, 62, 44, 40, 27, 32, 83, 48, 48, 39, 55, 22, 7, 11, 16, 39]
[40, 51, 44, 10, 73, 20, 12, 38, 7, 64, 28, 80, 70, 39, 44, 62]
[10, 27, 45, 19, 63, 49, 50, 38, 83, 80, 68, 30, 64, 17, 37, 73]
[72, 59, 6, 31, 69, 59, 56, 89, 29, 41, 42, 18, 21, 46, 62, 37]
[59, 60, 36, 35, 63, 47, 43, 30, 46, 75, 52, 36, 31, 66, 44, 35]
[43, 48, 49, 17, 15, 30, 33, 36, 21, 72, 44, 71, 43, 20, 45, 65]
[62, 66, 61, 81, 44, 53, 59, 32, 48, 22, 36, 31, 47, 21, 21, 56]
[80, 11, 29, 99, 59, 21, 70, 66, 70, 34, 36, 10, 31, 41, 67, 40]
[28, 62, 54, 87, 10, 50, 48, 40, 52, 21, 54, 38, 42, 36, 57, 96]
[45, 82, 20, 39, 49, 18, 49, 67, 34, 52, 20, 34, 72, 97, 32, 30]
[15, 64, 34, 86, 53, 22, 12, 50, 32, 40, 11, 40, 39, 12, 37, 83]
[62, 47, 43, 19, 20, 70, 9, 52, 19, 65, 16, 62, 99, 60, 24, 42]
[56, 26, 20, 62, 58, 42, 45, 60, 27, 63, 66, 31, 49, 53, 18, 66]
[29, 26, 44, 34, 47, 52, 47,

In [27]:
with open('./test_set/{}_{}agents.pth'.format('random64', 32), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[51, 50, 16, 56, 56, 56, 30, 30, 33, 8, 16, 17, 55, 43, 15, 45, 47, 63, 59, 78, 54, 70, 90, 53, 24, 102, 18, 17, 34, 21, 10, 36]
[42, 71, 42, 49, 44, 92, 51, 68, 70, 22, 82, 49, 22, 48, 67, 46, 20, 89, 25, 34, 89, 26, 22, 28, 48, 16, 63, 22, 3, 21, 26, 16]
[61, 49, 71, 23, 64, 75, 74, 50, 46, 46, 73, 63, 55, 37, 7, 81, 39, 27, 23, 14, 59, 31, 46, 57, 51, 9, 48, 76, 23, 92, 48, 49]
[72, 49, 63, 60, 57, 57, 45, 31, 16, 45, 61, 39, 64, 29, 28, 89, 14, 38, 61, 28, 62, 37, 71, 63, 35, 16, 65, 34, 15, 13, 8, 62]
[24, 61, 61, 20, 33, 24, 55, 32, 47, 22, 37, 75, 53, 77, 27, 40, 48, 62, 70, 73, 83, 23, 13, 25, 71, 95, 40, 10, 76, 37, 41, 35]
[103, 41, 15, 32, 28, 31, 45, 32, 34, 57, 47, 55, 44, 23, 59, 37, 28, 45, 42, 64, 79, 79, 77, 27, 12, 12, 55, 86, 26, 20, 17, 68]
[43, 54, 14, 56, 51, 84, 27, 69, 89, 55, 13, 67, 44, 31, 85, 18, 47, 19, 20, 59, 29, 43, 53, 57, 61, 24, 37, 50, 18, 27, 73, 16]
[58, 38, 72, 31, 50, 5, 26, 16, 6, 67, 73, 102, 70, 40, 98, 21, 56, 11, 19, 33, 34, 59, 26, 14, 41, 

In [28]:
with open('./test_set/{}_{}agents.pth'.format('random32', 4), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[21, 11, 47, 14]
[38, 10, 14, 15]
[35, 56, 33, 6]
[10, 8, 43, 27]
[8, 21, 25, 27]
[21, 14, 24, 32]
[23, 28, 24, 22]
[29, 28, 19, 13]
[27, 33, 21, 23]
[3, 13, 33, 13]
[11, 7, 19, 15]
[25, 17, 45, 16]
[9, 35, 15, 8]
[30, 7, 35, 28]
[22, 11, 29, 9]
[13, 3, 3, 27]
[13, 11, 29, 23]
[28, 47, 44, 17]
[22, 17, 24, 17]
[17, 16, 39, 24]


In [29]:
with open('./test_set/{}_{}agents.pth'.format('random32', 8), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[25, 23, 23, 16, 21, 16, 21, 8]
[24, 23, 46, 44, 56, 43, 20, 33]
[9, 20, 31, 8, 41, 34, 12, 18]
[35, 13, 25, 30, 30, 18, 28, 17]
[35, 40, 12, 9, 17, 41, 42, 13]
[18, 52, 35, 9, 38, 21, 31, 19]
[13, 21, 18, 13, 10, 34, 23, 27]
[33, 11, 23, 8, 8, 31, 6, 37]
[20, 24, 24, 9, 4, 11, 45, 27]
[42, 10, 51, 2, 40, 19, 4, 23]
[10, 8, 36, 27, 1, 23, 21, 19]
[29, 53, 24, 34, 2, 20, 15, 11]
[11, 24, 12, 17, 29, 22, 22, 35]
[27, 19, 42, 38, 8, 26, 13, 15]
[24, 22, 32, 11, 20, 26, 28, 37]
[39, 40, 24, 7, 27, 27, 32, 13]
[18, 6, 29, 19, 18, 3, 18, 22]
[28, 35, 24, 16, 16, 34, 21, 18]
[13, 17, 12, 17, 41, 14, 4, 27]
[4, 51, 46, 34, 25, 12, 37, 33]


In [30]:
with open('./test_set/{}_{}agents.pth'.format('random32', 16), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[25, 29, 37, 50, 31, 26, 14, 11, 23, 14, 14, 40, 35, 5, 21, 10]
[11, 16, 45, 41, 25, 38, 34, 43, 36, 10, 8, 35, 39, 21, 19, 29]
[13, 65, 4, 17, 45, 14, 29, 34, 11, 19, 41, 34, 25, 20, 21, 39]
[22, 33, 28, 24, 10, 17, 36, 31, 45, 30, 2, 47, 29, 26, 12, 27]
[5, 15, 24, 23, 23, 3, 30, 39, 3, 27, 38, 11, 39, 25, 23, 21]
[34, 24, 53, 30, 30, 26, 16, 15, 2, 31, 29, 31, 22, 37, 27, 21]
[18, 20, 28, 18, 31, 13, 26, 16, 11, 17, 15, 12, 16, 16, 21, 21]
[9, 19, 33, 26, 17, 13, 26, 35, 18, 24, 16, 44, 17, 22, 15, 22]
[15, 22, 28, 8, 32, 20, 18, 7, 22, 12, 5, 39, 17, 29, 21, 26]
[24, 32, 15, 14, 42, 6, 13, 7, 23, 10, 21, 40, 27, 14, 10, 31]
[13, 53, 3, 22, 34, 28, 15, 13, 9, 20, 34, 13, 9, 6, 8, 21]
[19, 26, 23, 5, 17, 20, 32, 8, 27, 45, 21, 22, 26, 31, 21, 13]
[37, 16, 24, 6, 38, 30, 10, 11, 24, 32, 17, 23, 33, 31, 7, 30]
[33, 31, 42, 12, 35, 15, 28, 17, 6, 22, 6, 32, 20, 39, 11, 16]
[17, 16, 8, 9, 29, 20, 41, 41, 20, 19, 20, 16, 9, 16, 8, 24]
[3, 14, 8, 38, 25, 9, 28, 42, 5, 17, 24, 35, 37, 24, 3

In [31]:
with open('./test_set/{}_{}agents.pth'.format('random32', 32), 'rb') as f:
    tests = pickle.load(f)

all_step_lists = simulate_all_tests(tests[30:50], 256)

[19, 25, 17, 40, 17, 4, 12, 17, 9, 17, 41, 15, 25, 21, 56, 8, 34, 18, 18, 11, 38, 37, 17, 9, 23, 6, 3, 23, 40, 15, 36, 21]
[45, 26, 30, 35, 46, 34, 26, 80, 19, 36, 53, 13, 33, 7, 38, 43, 19, 17, 38, 7, 26, 22, 23, 15, 19, 24, 30, 7, 23, 30, 24, 53]
[68, 37, 32, 39, 16, 42, 16, 20, 18, 15, 23, 23, 63, 29, 85, 20, 21, 32, 14, 74, 13, 13, 18, 12, 8, 14, 42, 8, 20, 6, 21, 15]
[48, 22, 9, 43, 23, 31, 29, 49, 19, 70, 22, 33, 17, 38, 35, 36, 47, 2, 8, 15, 23, 49, 30, 18, 22, 5, 29, 16, 42, 11, 25, 30]
[32, 23, 39, 20, 39, 33, 40, 15, 33, 44, 30, 34, 27, 21, 29, 33, 20, 22, 19, 13, 26, 2, 15, 25, 14, 20, 31, 27, 27, 30, 16, 10]
[27, 34, 24, 28, 65, 13, 23, 30, 12, 45, 39, 50, 21, 6, 27, 18, 16, 39, 21, 29, 24, 17, 9, 24, 4, 10, 11, 17, 20, 34, 18, 19]
[18, 26, 4, 19, 23, 11, 31, 13, 18, 8, 42, 18, 3, 19, 13, 15, 15, 38, 29, 9, 4, 28, 35, 26, 14, 19, 10, 17, 14, 44, 28, 33]
[21, 61, 32, 19, 9, 26, 32, 30, 32, 25, 35, 43, 17, 44, 18, 20, 6, 34, 32, 16, 20, 9, 4, 34, 89, 5, 25, 52, 27, 21, 12, 20